In [ ]:
import pandas as pd
import geopandas as gpd
# from jupyterthemes import jtplot
import matplotlib.pyplot as plt
import plotly.express as px
import basedosdados as bd

# jtplot.style(theme='monokai', ticks=True) (Essa maneira de usar temas também não funciona com esse tipo de função de mapa por algum motivo

Primeiramente, vamos criar uma tabela onde cada linha contém um ID de município e a sigla do estado ao qual esse município pertence, usando como apoio uma base do pacote "Base dos Dados".

In [ ]:
states = bd.read_sql(
    '''
    SELECT DISTINCT id_municipio AS id, sigla_uf
    FROM  basedosdados.br_bd_diretorios_brasil.setor_censitario AS censo
    GROUP BY id_municipio, sigla_uf
    ''',
    billing_project_id='geobr-analysis')

cities = gpd.read_file("geojson\\brasil.json", encoding='utf-8')

In [ ]:
states.head()

In [ ]:
cities.head()

Agora, daremos merge dessa tabela de estados com a que contém as informações geográficas de cada município, nos permitindo filtrar os dados de geometria de um estado específico, nesse caso, o do Rio de Janeiro.

In [ ]:
brasil_data = pd.merge(states, cities, on='id')
brasil_data = gpd.GeoDataFrame(brasil_data, geometry='geometry')

print(brasil_data)

In [ ]:
RJ_data = brasil_data[brasil_data["sigla_uf"] == "RJ"].reset_index(drop=True)
RJ_data = RJ_data.astype({"id": int})

In [ ]:
covid = pd.read_csv('caso_full.csv')
covid.rename(columns={"city_ibge_code": "id", "last_available_confirmed_per_100k_inhabitants": "cases_per_100k", "last_available_death_rate": "death_rate"}, inplace=True)
covid = covid[covid["id"] > 1000].astype({"id": int})
covid_rj = covid[(covid["state"] == "RJ") & (covid["date"] == "2021-11-17")].reset_index(drop=True)

In [ ]:
covid_rj = covid_rj[["id", "estimated_population", "cases_per_100k", "death_rate"]]

In [ ]:
RJ_fulldata = pd.merge(RJ_data, covid_rj, on='id')
RJ_fulldata

In [ ]:
fig = px.choropleth(RJ_fulldata,
                   geojson = RJ_fulldata["geometry"],
                   locations = RJ_fulldata.index,
                   color = "cases_per_100k",
                   hover_name = "name",
                   hover_data = ["cases_per_100k", "death_rate"],
                   labels = {
                       "cases_per_100k": "Cases per 100k habitants",
                       "death_rate": "Mortality rate"
                   },
                   color_continuous_scale = "sunsetdark"
)

fig.update_geos(
                projection=dict(
                    scale=55
                    ),
                center = dict(
                    lat=-22.208333,
                    lon=-42.896388
                    )
)

fig.show()